<a href="https://colab.research.google.com/github/sreeman-11021996/CloudxLab_Fashion_Recommendation_system/blob/main/test_transfer_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [16]:
!pip install tensorflow-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 588.3 MB 6.2 kB/s 
     |████████████████████████████████| 6.0 MB 40.6 MB/s 
     |████████████████████████████████| 1.7 MB 40.4 MB/s 
     |████████████████████████████████| 439 kB 45.8 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.12:
      Successfully 

In [1]:
!nvidia-smi

Sat Nov 26 12:58:36 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [116]:
!pip install import_ipynb
import import_ipynb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.6 MB 3.9 MB/s 


In [19]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [20]:
!kaggle datasets download -d paramaggarwal/fashion-product-images-small

100% 564M/565M [00:22<00:00, 29.8MB/s]
100% 565M/565M [00:22<00:00, 26.8MB/s]


In [21]:
import zipfile
zip_ref = zipfile.ZipFile("/content/fashion-product-images-small.zip", "r")
zip_ref.extractall("/content")
zip_ref.close()

In [57]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow import keras 
from keras.applications.resnet import preprocess_input

In [117]:
from utils import clean_data_

In [56]:
tensorflow.__version__

'2.9.2'

In [120]:
data=pd.read_csv('/content/styles.csv',usecols=["id"])

In [121]:
data.head()

,id
0,15970
1,39386
2,59263
3,21379
4,53759


In [122]:
data = clean_data_(data)

[INFO]: Checking if all images are present


44446it [00:03, 12037.27it/s]

[INFO]: Dropping indices: [6697, 16207, 32324, 36399, 40022]


In [123]:
data.shape

(44441, 1)

In [124]:
data["id"] = "images/" + data["id"].astype(str) + ".jpg"

In [125]:
data.head()

,id
0,images/15970.jpg
1,images/39386.jpg
2,images/59263.jpg
3,images/21379.jpg
4,images/53759.jpg


In [126]:
from PIL import Image

In [133]:
def extract_images(img_path):
    # load the image
    img = Image.open(img_path[0])
    img_resized = img.resize((224,224))
    # img -> array
    img_array = np.array(img_resized)
    # keras only takes images in batches
    expanded_img_array = np.expand_dims(img_array,axis=0)
    # preprocess img according to ResNet50
    processed_img = preprocess_input(expanded_img_array)
    
    return processed_img

In [148]:
data_sample = pd.DataFrame()

In [150]:
data_sample = data[:2].apply(extract_images,axis=1)

In [151]:
data_sample

0    [[[[151.061   138.22101 131.32   ], [151.061  ...
1    [[[[151.061   138.22101 131.32   ], [151.061  ...
dtype: object

In [2]:
import os

In [13]:
root_dir = "drive/MyDrive/Fashion_Recommendation_system/transfer_learning_model/transfer_learning_model_files"
model_json = os.path.join(root_dir,"myModel_arc_transfer.json")
model_wts =  os.path.join(root_dir,"myModel_wts_transfer.h5")

In [5]:
from keras.models import model_from_json

In [6]:
# get the model

In [14]:
file = open(model_json)
loaded_model = file.read()
file.close()

In [15]:
loaded_model_final = model_from_json(loaded_model)
loaded_model_final.load_weights(model_wts)